In [1]:
# from dotenv import load_dotenv
# import os
import chromadb
# from pathlib import Path

#.env file load
# load_dotenv('../RAG_Pipeline/.env')


# HOST = os.getenv('CHROMADB_HOST')
# PORT = os.getenv('CHROMADB_PORT')

client = chromadb.HttpClient(host='mywol.iptime.org', port=3307)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [38]:
collection_name = f"document_A"
collection = client.get_or_create_collection(collection_name)
print(collection.get(include=['metadatas', 'documents']))
# print(collection.get())
client.delete_collection(name=collection_name)

{'ids': ['chunk_0', 'chunk_1', 'chunk_2', 'chunk_3', 'chunk_4', 'chunk_5', 'chunk_6', 'chunk_7', 'chunk_8', 'chunk_9', 'chunk_10', 'chunk_11', 'chunk_12', 'chunk_13', 'chunk_14', 'chunk_15', 'chunk_16', 'chunk_17', 'chunk_18', 'chunk_19', 'chunk_20', 'chunk_21', 'chunk_22', 'chunk_23', 'chunk_24', 'chunk_25', 'chunk_26', 'chunk_27', 'chunk_28', 'chunk_29', 'chunk_30', 'chunk_31', 'chunk_32', 'chunk_33', 'chunk_34', 'chunk_35'], 'embeddings': None, 'metadatas': [{'page': 1}, {'page': 0}, {'page': 0}, {'page': 3}, {'page': 4}, {'page': 6}, {'page': 0}, {'page': 0}, {'page': 0}, {'page': 8}, {'page': 9}, {'page': 0}, {'page': 0}, {'page': 11}, {'page': 0}, {'page': 0}, {'page': 12}, {'page': 0}, {'page': 0}, {'page': 14}, {'page': 0}, {'page': 0}, {'page': 15}, {'page': 0}, {'page': 0}, {'page': 16}, {'page': 0}, {'page': 0}, {'page': 18}, {'page': 19}, {'page': 0}, {'page': 0}, {'page': 0}, {'page': 21}, {'page': 22}, {'page': 23}], 'documents': ['[Page 1]\n\n한화생명 간편가입 e시그니처암보험 무배당 상품요약서

In [41]:
import re

# 텍스트 전처리 함수
def clean_text(text):
    """텍스트 전처리 개선"""
    # 페이지 번호 제거
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\s*\d+\s*\n', '', text)

    # 불필요한 특수문자 제거
    text = re.sub(r'[•◆▶▷→▪]', '', text)

    # 삭제하고 싶은 특정 문구들
    remove_patterns = [
        r'도로교통법\s*\n',
        r'.*법제처.*\n',
        r'국가법령정보센터\s*\n',
        r'confusion matrix.*?(?=\n|$)',  # confusion matrix 관련 텍스트
        r'[rR][oO][cC].*?(?=\n|$)',     # ROC 관련 텍스트
    ]

    # 모든 패턴을 하나의 정규식으로 결합
    combined_pattern = '|'.join(remove_patterns)
    text = re.sub(combined_pattern, '', text)

    # 불필요한 숫자 패턴 제거
    text = re.sub(r'^\d+\.\s*', '', text, flags=re.MULTILINE)

    # 연속된 공백 정리
    text = re.sub(r'\s+', ' ', text)

    # 불필요한 줄바꿈 정리
    text = re.sub(r'\n+', '\n', text)

    # URL 패턴 정리
    text = re.sub(r'http\S+\s?', '', text)

    return text.strip()

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2', # 한국어 자연어 추론에 최적화된 모델
        model_kwargs={'device':'cpu'}, # 모델이 CPU에서 실행되도록 설정 GPU를 사용할 수 있다면 cuda로 설정
        encode_kwargs={'normalize_embeddings':True}, # 임베딩을 정규화하여 모든 벡터가 같은 범위 값을 갖도록 한다.
    )

In [10]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_path = '../docs/A 한화생명 간편가입 암보험.pdf'
loader = PyMuPDFLoader(pdf_path)
pages = loader.load() # 페이지 별 추출 텍스트를 리스트로 반환
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기
    chunk_overlap=100,      # 청크 간 중복
    length_function=len,
    separators=["\n\n"] # 분할 기준
)

for i, page in enumerate(pages):
    print(f"{i}번쨰")
    print(f"{page.metadata}")
    print(f"{page.page_content}\n\n")

# for i, page in enumerate(pages):
#     text = clean_text(page.page_content)

#     chunk = text_splitter.split_text(text)
#     print(chunk[0])

#     embedding = embeddings_model.embed_query(chunk[0])
#     print(embedding)

#     collection.add(
#         embeddings = [embedding],
#         documents = [chunk[0]],
#         ids = [f"chunk_{i}"]
#     )

0번쨰
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-10T09:07:51+09:00', 'source': '../docs/A 한화생명 간편가입 암보험.pdf', 'file_path': '../docs/A 한화생명 간편가입 암보험.pdf', 'total_pages': 23, 'format': 'PDF 1.5', 'title': '【별표 】', 'author': 'a', 'subject': '', 'keywords': '', 'moddate': '2024-01-10T09:07:51+09:00', 'trapped': '', 'modDate': "D:20240110090751+09'00'", 'creationDate': "D:20240110090751+09'00'", 'page': 0}
한화생명 간편가입 e시그니처암보험 무배당 
상품요약서 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
이 상품요약서는 보험약관 등 한화생명 간편가입 e시그니처암보험 무배당의 기초서류에 기재된 주요내
용을 요약한 것이므로 구체적인 내용은 반드시 보험약관 및 상품설명서 등을 참조하시기 바랍니다


1번쨰
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-10T09:07:51+09:00', 'source': '../docs/A 한화생명 간편가입 암보험.pdf', 'file_path': '../docs/A 한화생명 간편가입 암보험.pdf', 'total_pages': 23, 'format': 'PDF 1.5', 'title': '【별표 】', 'author': 'a', 'subject': '', 'keywords': '', 'moddate': '2024-01-10T09:07

In [7]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


pdf_path = '/Users/antaemin/Desktop/RAG_Pipeline_Exam_Gnew_GitLab/RAG_Pipeline/docs/A 한화생명 간편가입 암보험.pdf'
loader = PyMuPDFLoader(pdf_path)
pages = loader.load() # 페이지 별 추출 텍스트를 리스트로 반환
print(pages)

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-10T09:07:51+09:00', 'source': '/Users/antaemin/Desktop/RAG_Pipeline_Exam_Gnew_GitLab/RAG_Pipeline/docs/A 한화생명 간편가입 암보험.pdf', 'file_path': '/Users/antaemin/Desktop/RAG_Pipeline_Exam_Gnew_GitLab/RAG_Pipeline/docs/A 한화생명 간편가입 암보험.pdf', 'total_pages': 23, 'format': 'PDF 1.5', 'title': '【별표 】', 'author': 'a', 'subject': '', 'keywords': '', 'moddate': '2024-01-10T09:07:51+09:00', 'trapped': '', 'modDate': "D:20240110090751+09'00'", 'creationDate': "D:20240110090751+09'00'", 'page': 0}, page_content='한화생명 간편가입 e시그니처암보험 무배당 \n상품요약서 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n이 상품요약서는 보험약관 등 한화생명 간편가입 e시그니처암보험 무배당의 기초서류에 기재된 주요내\n용을 요약한 것이므로 구체적인 내용은 반드시 보험약관 및 상품설명서 등을 참조하시기 바랍니다'), Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-10T09:07:51+09:00', 'source'

In [7]:
# 사용자 질문 반환 함수
def get_user_query():
    lines = []

    while True:
        line = input("질문을 입력하세요 (종료: 빈 줄에서 Enter)")
        # 빈 줄이고 이전에 입력된 내용이 있으면 종료
        if not line and lines:
            break
        # 입력된 줄이 있으면 추가
        if line:
            lines.append(line)

    return "\n".join(lines)

text = get_user_query()
print(text)

abcd
abdd
awws


In [59]:
user_query = get_user_query()

user_query_embedding = embeddings_model.embed_query(user_query)
k = 2

results = collection.query(
    query_embeddings=[user_query_embedding],
    n_results=k,
    include=['documents']
)
print(user_query)

for i, chunk_id in enumerate(results['ids'][0]):
    print(f"{i}번째 청크 ID: {chunk_id}")
    print(f"해당 청크 내용: {results['documents'][0][i]}")  # 내용 앞부분만
    print('-------\n')


무인 교통단속용 장비의 설치 및 관리
0번째 청크 ID: chunk_6
해당 청크 내용: 제4조의2(무인 교통단속용 장비의 설치 및 관리) ① 지방경찰청장, 경찰서장 또는 시장등은 이 법을 위반한 사실을 기 록ㆍ증명하기 위하여 무인(無人) 교통단속용 장비를 설치ㆍ관리할 수 있다. ② 무인 교통단속용 장비의 철거 또는 원상회복 등에 관하여는 제3조제3항부터 제5항까지의 규정을 준용한다. 이 경우 "교통안전시설"은 "무인 교통단속용 장비"로 본다. [전문개정 2011. 6. 8.] 제4조의2(무인 교통단속용 장비의 설치 및 관리) ① 지방경찰청장, 경찰서장 또는 시장등은 이 법을 위반한 사실을 기 록ㆍ증명하기 위하여 무인(無人) 교통단속용 장비를 설치ㆍ관리할 수 있다. ② 무인 교통단속용 장비의 철거 또는 원상회복 등에 관하여는 제3조제4항부터 제6항까지의 규정을 준용한다. 이 경우 "교통안전시설"은 "무인 교통단속용 장비"로 본다.<개정 2018. 6. 12.> [전문개정 2011. 6. 8.] [시행일 : 2019. 6. 13.] 제4조의[시행일 : 2019. 6. 13] 제4조의제5조(신호 또는 지시에 따를 의무) ① 도로를 통행하는 보행자와 차마의 운전자는 교통안전시설이 표시하는 신호 또 는 지시와 다음 각 호의 어느 하나에 해당하는 사람이 하는 신호 또는 지시를 따라야 한다. <개정 2015. 7. 24.> 교통정리를 하는 국가경찰공무원(의무경찰을 포함한다. 이하 같다) 및 제주특별자치도의 자치경찰공무원(이하 "자치경찰공무원"이라 한다) 국가경찰공무원 및 자치경찰공무원(이하 "경찰공무원"이라 한다)을 보조하는 사람으로서 대통령령으로 정하는 사람(이하 "경찰보조자"라 한다) ② 도로를 통행하는 보행자와 모든 차마의 운전자는 제1항에 따른 교통안전시설이 표시하는 신호 또는 지시와 교통 정리를 하는 국가경찰공무원ㆍ자치경찰공무원 또는 경찰보조자(이하 "경찰공무원등"이라 한다)의 신호 또는 지시 가 서로 다른 경우에는 경찰공무원등의 신호 또는 지시에 따라야 한다.

In [61]:
import httpx

contexts = []

for i, doc in enumerate(results['documents'][0]):
    contexts.append(f"{i + 1}번째 관련 문서 : \n{doc}\n")

context_text = "\n".join(contexts)

# 프롬프트 생성
prompt = f"""질문: {user_query}
참고할 도로교통법 내용 :
{context_text}

위 도로교통법 내용만을 참고하여 질문에 답변해주시기 바랍니다.
다른 참고 문서는 참고하지 말아주시기 바랍니다.
"""
api_key = UPSTAGE_KEY_TM
base_url = "https://api.upstage.ai/v1/chat/completions"
payload = {
    "model": "solar-pro",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "stream": False
}

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# timeout -> context가 길어 5초로는 부족, 60초로 늘려줌
with httpx.Client(timeout=60.0) as client:
    response = client.post(base_url, headers=headers, json=payload)

# 응답 결과 출력
print(response.status_code)
print(response.json())

200
{'id': '52a3bc67-ed82-4a63-ade0-055299888f94', 'object': 'chat.completion', 'created': 1752641648, 'model': 'solar-pro-250422', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '무인 교통단속용 장비의 설치 및 관리에 대해 문의하셨습니다. 도로교통법 제4조의2 ①항에 따라 지방경찰청장, 경찰서장 또는 시장 등은 교통법규 위반 사실을 기록하고 증명하기 위해 무인 교통단속용 장비를 설치 및 관리할 수 있습니다. 또한, 무인 교통단속용 장비의 철거 또는 원상회복 등에 대해서는 제3조 제4항부터 제6항까지의 규정을 준용하며, 이 경우 교통안전시설은 무인 교통단속용 장비로 간주됩니다.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1834, 'completion_tokens': 113, 'total_tokens': 1947}, 'system_fingerprint': None}


In [8]:
data = response.json()

print(data['choices'][0]['message']['content'])

NameError: name 'response' is not defined